In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns

from plotly.subplots import make_subplots

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
df_users = pd.read_csv("[뱅크샐러드] DS 사전과제 Dataset/df_users.csv")
df_user_visits = pd.read_csv("[뱅크샐러드] DS 사전과제 Dataset/df_user_visits.csv")

In [ ]:
df_users.describe().T.join(pd.DataFrame(df_users.dtypes)).rename(columns={0:"dtypes"})

In [ ]:
df_user_visits.describe().T.join(pd.DataFrame(df_user_visits.dtypes)).rename(columns={0:"dtypes"})

# Feature engineering

In [ ]:
df_users["date_joined"] = pd.to_datetime(df_users["date_joined"])
df_user_visits["avg_time_to_interactive"] = df_user_visits["avg_time_to_interactive"].fillna(0)

In [ ]:
joined_df = pd.merge(df_users, df_user_visits, left_on="user_uuid", right_on="user_uuid")

joined_df["year_month_joined"]= joined_df["date_joined"].dt.to_period("M")
joined_df["dayofweek_joined"] = joined_df["date_joined"].dt.dayofweek # monday = 0 

joined_df.set_index("date_joined", inplace=True)
joined_df.sort_index(inplace=True)

In [ ]:
joined_df.head()

group_1 의 모든 사용자의 방문횟수가 group_2보다 2배정도 많은데 애초에 사용자수가 2배 많음.
- 인당 평균 방문수는 비슷.
- visit 0 인 사람도 대략 group_1 = 2group_2, 그룹 사이즈의 차이로 보임. -> 어느 한 그룹이 visit 0 의 영향이 없음.

# 사용자 그룹 간 앱 방문 비교 분석

## group_1 과 group_2 의 사용자 방문 수의 차이가 있는지 검증

In [ ]:
joined_df.groupby(["group"]).agg({
    "visits":{"count", sum, np.mean,np.median, np.std, min, max},
    "avg_time_to_interactive":{np.mean, np.median, np.std, min, max}
})

아래 Histogram 을 보면 그륩별 방문 수 분포는 비슷한 shape 을 그리고 있습니다.

In [ ]:
group1 = joined_df.loc[joined_df["group"] == "group_1"].copy()
group2 = joined_df.loc[joined_df["group"] == "group_2"].copy()

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=["group1", "group2"]    
)

fig.add_trace(
    go.Histogram(
        name = "group1",
        x = group1["visits"]
    ), row=1, col=1
)
fig.add_trace(
    go.Histogram(
        name = "group2",
        x = group2["visits"]
    ), row=2, col=1
)

fig.update_layout(
    title = "group 별 방문 수 분포"
)
fig.show()

In [ ]:
group1_sample = group1.sample(100)
group2_sample = group2.sample(100)

In [ ]:
print(f"group1_sample mean    = {group1_sample['visits'].mean()}")
print(f"group1_sample std_dev = {group1_sample['visits'].std()}")

In [ ]:
print(f"group2_sample mean    = {group2_sample['visits'].mean()}")
print(f"group2_sample std_dev = {group2_sample['visits'].std()}")

## 그룹별 통신사별 방문 수 차이

group1 과 group2의 사용자 방문 수의 차이가 통신사 별로 다른지? -> ex: A 통신사를 쓰는 group1는 A통신사를 쓰는 grouop2 사용자보다 방문 수가 많은가?

In [ ]:
joined_df.groupby(["carrier", "group"]).agg({
    "visits":{"count", sum, np.mean, np.std, min, max},
    "avg_time_to_interactive":{np.mean, np.median, np.std, min, max}
})

### 그룹별 통신사별 방문 수 분포

In [ ]:
carriers = ["노랑", "보라", "초록"]
groups   = ["group_1", "group_2"]

df_dict = {}
for carrier in carriers:
    for group in groups:
        df = joined_df.loc[(joined_df["carrier"] == carrier) &
                           (joined_df["group"] == group)].copy()
        df_dict[carrier +"_"+ group] = df

In [ ]:
fig = make_subplots(
    rows=3, 
    cols=2,
    subplot_titles=list(df_dict.keys())
)

for idx, (name, df) in enumerate(df_dict.items()):
    fig.add_trace(
        go.Histogram(
            name = name,
            x = df["visits"]
        ),
        row=(idx//2)+1, col= (idx%2)+1
    )
    fig.update_xaxes(range=[0,100])
    
fig.update_layout(
    title = "통신사별_group별 방문 수 분포"
)

fig.show()

### 가설검증

In [ ]:
# def TestDifference(carrier_name, n, c, g1, g2):
    

In [ ]:
# for carrier in ["노랑", "보라", "초록"]:
#     carrier_df = joined_df.loc[joined_df["carrier"] == carrier].copy()
    
#     g1 = carrier_df.loc[carrier_df["group"] == "group_1"].sample(100).copy()
#     g2 = carrier_df.loc[carrier_df["group"] == "group_2"].sample(100).copy()
    
#     TestDifference(n, 0.)

In [ ]:
n = 100

for name, df in df_dict.items():
    sample = df.sample(100)
    print(f"{name} mean    = {sample['visits'].mean()}")
    print(f"{name} std_dev = {sample['visits'].std()}")
    print("")

# 방문 수 분석 및 예측

## 다른 정보들과의 관계

In [ ]:
joined_df.head()

아래 pairplot 을 보면 2가지를 알수 있습니다.
1. 평균 앱 로딩 타임이 길어지면 visit 숫자가 줄어듭니다. 하지만 아주 약한 관계 -0.002
2. 총 앱 로딩 타임이 길어지면 평균 앱 로딩 타임도 길어집니다. -> 많이 방문하면 총 로딩 타임이 길어질것, 많이 방문하면 그만큼 서버장애나 다른이유요 인해 엄청 오래걸리때가 걸릴수 있음=> 평균을 높임(예상).

총 가입자수가 늘면서 앱로딩 시간이 길어지는지 보자. 그렇타면:
- 초반에만 사용한 사람들은 방문 수가 적을것 + 가입자가 적은 초반에는 로딩시간이 빨라서 평균로딩시간이 짧음
- 초반 ~ 현재까지 사용한 사람들은 방문수가 높을것 + 가입자수가 늘면서 평균로딩 시간도 오름/

문제: 초반에 가입했더라도 후반에만 사용했을수도 있다... 언제 사용했는지의 데이터가 없어서 불가능.

앱버젼별로 보면됨. --> 앱버젼도 평균 앱로딩 시간에 영향을 미치지않음.

In [ ]:
joined_df.groupby(["app_version_joined"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":{"mean", np.std, np.median}
})

In [ ]:
joined_df.groupby(["year_month_joined"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":"mean"
})

In [ ]:
joined_df.corr()

In [ ]:
sns.pairplot(joined_df, kind="scatter")
plt.show()

---

시간적으로 봤을때 두 그륩이 다른지 확인 -> ex: group_1 은 1월에 대부분 가입, group_2 는 3~4월에?
    - 두그룹 둘다 가입자수가 그륩의 비율에 동일하게 증가. -> group_1가입자 = 2group_2가입자.

In [ ]:
# fig = go.Figure()

# fig.add_trace(
#     go.Histogram(
#         name = "group1",
#         x= joined_df.loc[joined_df["group"] == "group_1"]["year_month_joined"]
#     )
# )

# fig.add_trace(
#     go.Histogram(
#         name = "group2",
#         x= joined_df.loc[joined_df["group"] == "group_2"]["year_month_joined"]
#     )
# )

# fig.show()

----

categorical column 들과 방문 수의 관계

- 가입 년-월 
- 그룹
- 통신사
- 앱 버젼

가입 년-월 분류:
- 2030년 1월 < 3월 < 4월 < 2월 순으로 평균 방문 수.
- 2030년 1월은 2,3,4월에 비해 평균 방문수가 낮고 평균 앱로딩 시간도 깁니다. -> 추가로 평균 앱로딩시간이 평균 방문 수의 영향을 미치는지 인과관계 분석을 해봐도 될것.

In [ ]:
joined_df.groupby(["year_month_joined"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":"mean"
})

In [ ]:
sns.boxplot(x=joined_df["year_month_joined"],
            y=joined_df["visits"],
            order=sorted(joined_df["year_month_joined"].unique()))

In [ ]:
joined_df.groupby(["group"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":"mean"
})

In [ ]:
sns.boxplot(x=joined_df["group"],
            y=joined_df["visits"])

In [ ]:
joined_df.groupby(["carrier"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":"mean"
})

In [ ]:
sns.boxplot(x=joined_df["carrier"],
            y=joined_df["visits"])

app_version_joined 분류:
- 가입한 버젼 별 평균 방문 수, 큰차이 없음.

In [ ]:
joined_df.groupby(["app_version_joined"]).agg({
    "user_uuid":"count",
    "visits":{"mean", "sum"},
    "avg_time_to_interactive":"mean"
})

In [ ]:
sns.boxplot(x=joined_df["app_version_joined"],
            y=joined_df["visits"])

## 방문 수 예측

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
joined_df.head()

In [ ]:
categorical_cols = ["carrier", "group", "app_version_joined"]
num_cols = ["sum_time_to_interactive", "avg_time_to_interactive"]

joined_df.drop(columns=["user_uuid"], inplace=True)
y = joined_df.pop("visits")
X = joined_df

In [ ]:
joined_df["year"] = joined_df["year_month_joined"].dt.year
joined_df["month"] = joined_df["year_month_joined"].dt.month

In [ ]:
joined_df.drop(columns=["year_month_joined"], inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)

stdscaler = StandardScaler()
X_train[num_cols] = stdscaler.fit_transform(X_train[num_cols])

In [ ]:
lin_reg = LinearRegression()

In [ ]:
lin_reg.fit(X_train, y_train)

In [ ]:
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
X_test[num_cols] = stdscaler.transform(X_test[num_cols])

In [ ]:
y_train_pred = lin_reg.predict(X_train)
y_pred = lin_reg.predict(X_test)

In [ ]:
print(f"train MSE = {mean_squared_error(y_train, y_train_pred)}")
print(f"test MSE = {mean_squared_error(y_test, y_pred)}")

In [ ]:
test_df = pd.concat([X_test, y_test], axis=1)

In [ ]:
test_df["y_pred"] = y_pred

In [ ]:
test_df

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=test_df.index,
        y=test_df["visits"],
        mode ="markers"
    )
)

fig.show()